In [1]:
import glob
import datetime
import random
import numpy as np
import pandas as pd

In [ ]:
# Combined Excel files
df = pd.DataFrame()

for f in glob.glob('Raw Data/Activities_*xlsx'):
    data = pd.read_excel(f)
    data['File'] = f
    df = df.append(data, ignore_index = True)

for f in glob.glob('Raw Data/Activities_*xls'):
    data = pd.read_excel(f)
    data['File'] = f
    df = df.append(data, ignore_index = True)

In [ ]:
df.to_excel('Combined_File.xlsx')

In [2]:
# Create Scatter Coordination Table
data = []
df = pd.read_excel('Activities.xlsx')

r = 5
X = lambda i, a: 0 if a == 'Traveling' else round(r*np.cos(np.pi/2 - i*(2*np.pi)/(len(df['Label'].unique())-1)), 3)
Y = lambda i, a: 0 if a == 'Traveling' else round(r*np.sin(np.pi/2 - i*(2*np.pi)/(len(df['Label'].unique())-1)), 3)

for i, a in enumerate(np.sort(np.delete(df['Label'].unique(), np.where(df['Label'].unique() == 'Traveling'), axis=0))):
    data.append([a, X(i, a), Y(i, a)])

data.append(['Traveling', 0, 0])    
df_coordinate = pd.DataFrame(data, columns = ['Activity', 'X', 'Y']).set_index('Activity')

In [3]:
df_coordinate.to_excel('Coordinate.xlsx')

In [8]:
# Resturcture Data for Power BI
data = []
df = pd.read_excel('Activities.xlsx')

for s in df['Student'].unique():
    df_s = df[df['Student'] == s]
    for h in range(0, 24):
        for m in range(0, 60, 15):
            time_index = [i for i, j in enumerate(df[df['Student'] == s]['Time']) if j <= datetime.time(h, m)]
            time_index = -1 if not time_index else max(time_index)
            data.append([s,                                                    # Student
                         datetime.time(h, m),                                  # Time
                         df_s.iloc[time_index, df.columns.get_loc('Label')],   # Activity
                         df_s.iloc[time_index, df.columns.get_loc('Pain')],    # Pain
                         df_s.iloc[time_index, df.columns.get_loc('Gain')],    # Gain
                         df_coordinate.loc[df_s.iloc[time_index, df.columns.get_loc('Label')], 'X'] + random.randrange(-220, 220, 1)/1000, # X
                         df_coordinate.loc[df_s.iloc[time_index, df.columns.get_loc('Label')], 'Y'] + random.randrange(-220, 220, 1)/1000  # Y
                        ])
            
df_timeline = pd.DataFrame(data, columns = ['Student', 'Time', 'Activity', 'Pain', 'Gain', 'X', 'Y'])

In [9]:
df_timeline.to_excel('Timeline.xlsx')

In [10]:
# Create Top Activity Table
df_count = df_timeline[['Time', 'Activity', 'Student']].groupby(['Time', 'Activity']).count().reset_index()
df_count['Percent'] = df_count['Student']/len(df['Student'].unique())*100

df_top = df_count[['Time', 'Student']].groupby(['Time']).max().reset_index()
df_top = pd.merge(df_top, df_count, on = ['Time', 'Student'])
df_top = df_top.drop(['Student', 'Percent'], axis = 1).drop_duplicates('Time')
#df_top = pd.merge(df_top, df_timeline[['Time', 'Activity', 'Pain', 'Gain']], on = ['Time', 'Activity'])